# ImageNet Classification
## Created by [Daniel Fojo](https://www.linkedin.com/in/daniel-fojo/) and [Janna Escur](https://www.linkedin.com/in/janna-escur-i-gelabert-276b1212b/?originalSubdomain=es) 
##[Postgraduate Course on Artificial Intelligence with Deep Learning](https://www.talent.upc.edu/ing/estudis/formacio/curs/310400/postgrau-artificial-intelligence-deep-learning/). 

We will first train a network to classify MNIST to get familiarized with PyTorch's syntax.


First, we define our neural network. In PyTorch,  NN are just Python classes that inherit from nn.Module. We declare all the layers that we are going to use in the constructor (init method), and then define how we should do the forward pass in the `forward` method. The `backward` method is done automatically, so we do not need to code it.

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from matplotlib.pyplot import imshow
if not torch.cuda.is_available():
    print("You should enable GPU runtime!!")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(in_features=4*4*50, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)  # Flatten while keeping the batch dimension
        x = F.relu(self.fc1(x))
        x = F.log_softmax(self.fc2(x), dim=1)
        return x
   

model = Net().to(device)  # Move model to GPU

We now define the dataloaders that will feed the data to our NN during trainig in batches. The `dataset` class `torchvision.dataset.MNIST` already has the MNIST dataset loaded. We could add data augmentation here.

In [0]:
train_dataset = datasets.MNIST("data/MNIST", train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),  # Here we could add extra data augmentation
                       transforms.Normalize((0.1307,), (0.3081,)),
                   ]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)

test_dataset = datasets.MNIST("data/MNIST", train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,)),
                   ]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

For our optimizer, we will use SGD, and it will optimize the parameters of our model. For our loss, we wil use Negative Log Likelihood (together with the log_softmax of the last layer it is equivalent to the Cross Entropy while being more numerically stable).

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.NLLLoss()

We will define a custom metric to monitor during training.

In [0]:
def accuracy(preds, y):
    preds = preds.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct = preds.eq(y.view_as(preds)).sum().item()
    return correct/preds.size(0)

Here, we do our training. At each step we do a forward pass of our input, we compute the gradients and we do an optimizer step. At the end of each epoch we also compute metrics with our validation dataset.

In [0]:
epochs = 3
for epoch in range(epochs):
    model.train()  # Train mode
    for i, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()  # Reset gradients from last step
        preds = model(x)  # Forward pass
        loss = criterion(preds, y)
        loss.backward()  # Compute gradients
        optimizer.step()  # Do an optimizer step
        if i%20 == 0:
            print(f"TRAIN Epoch {epoch}: [{i}/{len(train_loader)}] Loss: {loss.item():.4f}; Accuracy: {accuracy(preds, y)}")
        
    
    model.eval()  # Test mode
    for i, (x, y) in enumerate(test_loader):
        x, y = x.to(device), y.to(device)
        preds = model(x)
        loss = criterion(preds, y)
        if i%20 == 0:
            print(f"TEST Epoch {epoch}: [{i}/{len(test_loader)}] Loss: {loss.item():.4f}; Accuracy: {accuracy(preds, y)}")

## Exercice 1: train with CIFAR10

You should complete the following code to train a network with CIFAR10.

If you want, you can try to add data augmentation, dropout and weight decay to regularize the model.

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = 
        self.conv2 = 
        self.fc1 = 
        self.fc2 = 

    def forward(self, x):

        return x
   

model = Net().to(device)


train_dataset = datasets.CIFAR10("data/CIFAR10", train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),  # Here we could add extra data augmentation
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                   ]))
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True)

test_dataset = datasets.CIFAR10("data/CIFAR10", train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                   ]))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False)

optimizer = optim.SGD(model.parameters(), lr=0.1)
criterion = nn.NLLLoss()

epochs = 3
for epoch in range(epochs):
    model.train()  # Train mode
    for i, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()  # Reset gradients from last step
        preds = model(x)  # Forward pass
        loss = criterion(preds, y)
        loss.backward()  # Compute gradients
        optimizer.step()  # Do an optimizer step
        if i%20 == 0:
            print(f"TRAIN Epoch {epoch}: [{i}/{len(train_loader)}] Loss: {loss.item():.4f}; Accuracy: {accuracy(preds, y)}")
        
    
    model.eval()  # Test mode
    for i, (x, y) in enumerate(test_loader):
        x, y = x.to(device), y.to(device)
        preds = model(x)
        loss = criterion(preds, y)
        if i%20 == 0:
            print(f"TEST Epoch {epoch}: [{i}/{len(test_loader)}] Loss: {loss.item():.4f}; Accuracy: {accuracy(preds, y)}")

# Pretrained imagenet models

In both PyTorch and Keras there are many models pretrained with Imagenet. You can see the lists here: https://pytorch.org/docs/stable/torchvision/models.html 
and https://keras.io/applications/.
We will see how to use a pretrained VGG16 from PyTorch to predict images taken from the internet.


We first declare our model with `pretrained=True`, which will download the pretrained weights.

In [0]:
import torchvision.models as models

model = models.vgg16(pretrained=True)

We also load the 1000 imagenet labels in a Python dictionary.

In [0]:
import json
import ast
urllib.request.urlretrieve("https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt", "labels.json")
with open("labels.json") as f:
    labels = ast.literal_eval(f.read())
print(labels)

We can now download a sample image to test our network.

In [0]:
import urllib
from PIL import Image
urllib.request.urlretrieve("https://3c1703fe8d.site.internapcdn.net/newman/gfx/news/hires/2018/2-dog.jpg", "image.jpg")
%ls
pil_image = Image.open("image.jpg")
imshow(np.asarray(pil_image))

We should do some preprocessing before feeding the image to our network. The following transformations are the same that were used when training with Imagenet. Note that we have to add a batch dimension at the beginning, even if we are feeding just one image to the network.

In [0]:
preprocess = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
torch_image = preprocess(pil_image)
torch_image = torch_image.unsqueeze(0)  # Add batch dimension
print("Image shape: ", torch_image.shape)
print("Image values: ", torch_image)

Now we can get the predictions of our network. Note that a network trained with imagenet will have 1000 outputs. Here we only show the top 3 predictions and their probabilities.

In [0]:
output = model(torch_image)
print("Output shape: ", output.shape)
probs, idxs = output.topk(3)
print(probs[0])
print([labels[idx.item()] for idx in idxs[0]])

# Exercice 2
Do the prediction with other images using different networks and compare the results.  You can use the function following function predict_from_url. The list of pretrained PyTorch networks can be found here https://pytorch.org/docs/stable/torchvision/models.html 

In [0]:
def predict_from_url(url, model):
    urllib.request.urlretrieve(url, "image.jpg")
    pil_image = Image.open("image.jpg")
    imshow(np.asarray(pil_image))
    torch_image = preprocess(pil_image).unsqueeze(0)
    output = model(torch_image)
    probs, idxs = output.topk(3)
    print(probs[0])
    print([labels[idx.item()] for idx in idxs[0]])

predict_from_url("https://img.purch.com/h/1400/aHR0cDovL3d3dy5saXZlc2NpZW5jZS5jb20vaW1hZ2VzL2kvMDAwLzEwMi8xNjIvb3JpZ2luYWwvYmVlci1hbmQtd2hlYXQuanBn", model)